In [ ]:

# Assignment ML6: Tic-Tac-Toe with Q-learning (Reinforcement Learning)
# Educational, simple Q-learning agent vs a random opponent. Not optimized for speed.

import numpy as np
import random
from collections import defaultdict
import matplotlib.pyplot as plt

# Board representation: 0 empty, 1 agent (X), -1 opponent (O)
def available_actions(state):
    return [i for i, v in enumerate(state) if v == 0]

def check_winner(state):
    board = np.array(state).reshape(3,3)
    for s in [1, -1]:
        # rows, cols, diags
        if any(np.all(board[r,:]==s) for r in range(3)): return s
        if any(np.all(board[:,c]==s) for c in range(3)): return s
        if np.all(np.diag(board)==s) or np.all(np.diag(np.fliplr(board))==s): return s
    if 0 not in state: return 0  # draw
    return None

def state_to_key(state):
    return tuple(state)

# Q-table as defaultdict
Q = defaultdict(float)
alpha = 0.5
gamma = 0.9
epsilon = 0.2

def choose_action(state):
    actions = available_actions(state)
    if random.random() < epsilon:
        return random.choice(actions)
    # choose action with max Q-value
    qs = [Q[(state_to_key(state), a)] for a in actions]
    max_q = max(qs)
    max_actions = [a for a,q in zip(actions, qs) if q == max_q]
    return random.choice(max_actions)

def play_episode(train=True):
    state = [0]*9
    history = []
    while True:
        # agent move
        a = choose_action(state)
        state[a] = 1
        history.append((state_to_key(state), a))
        winner = check_winner(state)
        if winner is not None:
            reward = 1 if winner==1 else (0 if winner==0 else -1)
            break
        # opponent random move
        opp_actions = available_actions(state)
        if not opp_actions:
            winner = check_winner(state)
            reward = 1 if winner==1 else (0 if winner==0 else -1)
            break
        opp = random.choice(opp_actions)
        state[opp] = -1
        winner = check_winner(state)
        if winner is not None:
            reward = 1 if winner==1 else (0 if winner==0 else -1)
            break
    # Q-learning update for history (simple Monte-Carlo-like update)
    if train:
        for s,a in reversed(history):
            old = Q[(s,a)]
            Q[(s,a)] = old + alpha*(reward - old)
            reward = reward*gamma
    return winner

# Training
episodes = 5000
wins = {1:0, -1:0, 0:0}
for ep in range(episodes):
    w = play_episode(train=True)
    wins[w] += 1
    if (ep+1) % 1000 == 0:
        print(f"Episode {ep+1}, cumulative wins: {wins}")

print("Training complete. Stats:", wins)

# Evaluate agent vs random opponent
test_episodes = 1000
wins = {1:0, -1:0, 0:0}
for _ in range(test_episodes):
    w = play_episode(train=False)
    wins[w] += 1
print("Evaluation results (agent win=1, draw=0, loss=-1):", wins)
